### 생육 환경 최적화 경진대회
https://dacon.io/competitions/official/235897/overview/description

주제 : [Algorithm] 청경채 사진과 환경 데이터를 활용한 잎면적 예측 알고리즘 개발

 train [Folder] : 학습용 데이터셋 (1009장)

	│	├ CASE01 : 각 케이스 별 데이터

	│	│	├ image : 이미지 데이터

	│	│	├ meta : 환경 데이터

	│	│		└ 촬영 된 시각으로부터 1일동안 측정된 '내부온도', '내부습도', 'CO2', 'EC' 의 환경정보

	│	│	└ label.csv :

	│	│		└ img_name : 해당 이미지 파일명

	│	│		└ leaf_weight : 해당 이미지로부터 1일 후의 잎 면적 (중량)

	│	├ CASE02

	│	├ CASE03

	│	└ ...

 test [Folder] : 테스트용 데이터셋 (342장)

	│	├ image : 이미지 데이터

	│	├ meta : 환경 데이터

	│		└ 촬영 된 시각으로부터 1일동안 측정된 '내부온도', '내부습도', 'CO2', 'EC' 의 환경정보

	└ sample_submission.csv

		└ img_name : 해당 이미지 파일명

		└ leaf_weight : 해당 이미지로부터 1일 후의 잎 면적 (중량) 예측 값

멀티인풋을 다루는 문제의 어려움, 데이터 결측이 심해서 다루기 어려웠음 - > 이미지 데이터로만 1일후의 잎 면적을 예측

### 1. 데이터 살펴보기

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('./dataset/train/CASE01/meta/CASE01_01.csv')
print(np.shape(data))
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: './dataset/train/CASE01/meta/CASE01_01.csv'

In [ ]:
data['시간']

In [ ]:
data.info()

### 이미지 데이터 살펴보기

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import os
import math
from glob import glob
import pandas as pd
import cv2

In [ ]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    # 이미지와 라벨 빈 리스트 생성
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return img_path_list, label_list
    # 이미지데이터와 라벨 데이터 분리

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    #img_path_list.sort(key=lambda x:(x.split('/')[-1].split('.')[0]))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0])) 
    return img_path_list

In [ ]:
all_img_path, all_label = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

In [ ]:
img = cv2.imread(all_img_path[146])
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.show()

In [ ]:
cropped = image[50:-50, 100:-100]
the = 140
ret, thresh1 = cv2.threshold(cropped,the,255, cv2.THRESH_BINARY)
ret, thresh2 = cv2.threshold(cropped,the,255, cv2.THRESH_BINARY_INV)
ret, thresh3 = cv2.threshold(cropped,the,255, cv2.THRESH_TRUNC)
ret, thresh4 = cv2.threshold(cropped,the,255, cv2.THRESH_TOZERO)
ret, thresh5 = cv2.threshold(cropped,the,255, cv2.THRESH_TOZERO_INV)

titles =['Original','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [cropped,thresh1,thresh2,thresh3,thresh4,thresh5]

for i in range(6):
	plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
	plt.title(titles[i])
	plt.xticks([]),plt.yticks([])

plt.show()

### 3. PipeLine 구성

In [ ]:
# 라이브러리 호출
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

#CPU연산/GPU연산 결정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### 하이퍼파라미터세팅

In [ ]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':110,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

### 시드 고정

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오는 경로 함수

In [ ]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    # 이미지와 라벨 빈 리스트 생성
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return img_path_list, label_list
    # 이미지데이터와 라벨 데이터 분리

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0])) 
    return img_path_list

In [ ]:
all_img_path, all_label = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

### 검증데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
train_img_path, vali_img_path, train_label, vali_label = train_test_split(all_img_path, all_label, test_size=0.2, random_state=CFG['SEED'])

### 커스텀 데이터 세팅

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image
        img = cv2.imread(img_path)
        img_bgr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cropped = img_bgr[50:-50, 100:-100]
        ret, image = cv2.threshold(cropped, 120, 255, cv2.THRESH_BINARY)
        #이미지를 자르고, 이진화해서 전달
        if self.transforms is not None:
            image = self.transforms(image)
        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
                    transforms.RandomVerticalFlip(0.2), # 상하 대칭
                    #transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])
test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])

In [ ]:
# Get Dataloader
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 레이어 생성

In [ ]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(
            #[8,3,256,256] -> [8,8,256,256]
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(),
            #[8,256,256] -> [8,128,128]
            nn.MaxPool2d(kernel_size=2, stride=2))
            
            #[8,8,256,256] -> [8,16,128,128]
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            #[16,128,128]-> [16,64,64]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,16,64,64] -> [8,32,64,64]
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            #[8,32,64,64]-> [8,32,32,32]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,32,32,32] -> [8,64,32,32]
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            #[8,64,32,32] -> [8,64,16,16]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,64,16,16] -> [8,128,16,16]
        self.layer5 = torch.nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            #[8,128,16,16] -> [8,128,7,7]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.regressor=nn.Linear(128*7*7,1)
        # 보통 여기서는 fc, full conneted network으로 불리는 선형출력
        # nn.Linear()는 입력의 차원, 출력의 차원 레이어를 만들어 본다.
        # 이런 레이어 대신 VGG나 ResNet으로하면 성능 향상이 더 있을까?

    def forward(self, x):
        # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
        # (Batch, 1, 256, 256)
        x = self.layer1(x)
        # (Batch, 8, 128, 128)
        x = self.layer2(x)
        # (Batch, 16, 64, 64)
        x = self.layer3(x)
        # (Batch, 32, 32, 32)
        x = self.layer4(x)
        # (Batch, 64, 16, 16)
        x = self.layer5(x)
        # (Batch, 128, 7, 7) -> Flatten (Batch, 128*7*7)
        x = torch.flatten(x, start_dim=1)
        # Regressor (Batch, 128*7*7) -> (Batch, 1)
        out = self.regressor(x)
        return out

In [ ]:
from torchsummary import summary
model = CNNRegressor().to(device)
summary(model,input_size=(3,256,256))

### 훈련 모델 생성

In [ ]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)
    # Loss Function
    criterion = nn.L1Loss().to(device)
    best_mae = 9999
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            loss = criterion(logit.squeeze(1), label)

            # 역전파 과정
            loss.backward() #역전파 시작
            optimizer.step() # 기울기업데이트
            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
        #scheduler는 Learning rate를 설정을 도와줌
        
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/best_model.pth')
            print('Model Saved.')
            #그리고 이때 손실함수의 기준은 train 데이터가 아닌 validation 데이터 기준

In [ ]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

### Optimizer

In [ ]:
model = CNNRegressor().to(device)

optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.RMSprop(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.Adamax(model.parameters(), lr=CFG["LEARNING_RATE"])
#optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["LEARNING_RATE"], weight_decay=0.0001)

#scheduler = None
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,lr_lambda=lambda epoch:0.95**epoch)
train(model, optimizer, train_loader, vali_loader, scheduler, device)

### 테스트 데이터 검증

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    # 학습할때 필요한 드롭아웃, 배치정규화등을 비활성화해서 추론만함
    
    model_pred = []
    with torch.no_grad():
    # 검증데이터에서 모델을 적용할때는 기울기를 추적하지 않음
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./saved/best_model.pth')
model = CNNRegressor().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)

### 제출

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./submit.csv', index=False)